# BUSINESS ANALYTICS

Project description.

Internship in the analytical department at Yandex.Afisha.

The task is to help optimize marketing expenses.

We have data from server logs with data on Yandex.Afisha visits from June 2017 through May 2018, dump file with all orders for the period, marketing expenses statistics.

We should find out
1. How people use the product
2. When the customer starts to buy
3. How much money each customer brings
4. When the customer pay off


Step 1. Download the data and prepare it for analysis

Store the data on visits, orders, and expenses in variables, optimizing the data for analysis and changing columns with the incorrect data type.


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
visits= pd.read_csv('/datasets/visits_log_us.csv',dtype={'Device': 'category'},
    parse_dates=['Start Ts', 'End Ts'])
orders=pd.read_csv('/datasets/orders_log_us.csv', parse_dates=['Buy Ts'])
costs=pd.read_csv('/datasets/costs_us.csv', parse_dates=['dt'])

In [ ]:
visits.head()

In [ ]:
visits.rename(columns={'Source Id':'source_id'}, inplace=True)


In [ ]:
visits.info()

Description of the data

The visits table (server logs with data on website visits):
Uid — user's unique identifier
Device — user's device
Start Ts — session start date and time
End Ts — session end date and time
Source Id — identifier of the ad source the user came from
All dates in this table are in YYYY-MM-DD format.


In [ ]:
orders.head()

The orders table (data on orders):
Uid — unique identifier of the user making an order
Buy Ts — order date and time
Revenue — Yandex.Afisha's revenue from the order
The costs table (data on marketing expenses):
source_id — ad source identifier
dt — date


In [ ]:
orders.info()

In [ ]:
orders.rename(columns={'Buy Ts':'buy_ts', 'Revenue':'revenue'}, inplace=True)
orders.head()

In [ ]:
costs.head()

costs — expenses on this ad source on this day

In [ ]:
costs.info()

In [ ]:
costs['source_id'].unique()

# 2. Make reports and calculate metrics.

1. Product


1.How many people use it every day, week, and month.
2.Sessions per day
3.Length of each session.
4.User retention rate

In [ ]:
#1.Let's look at the data on user activity. This metric tells how loyal the audience is — how often they return.
#DAU — the number of daily active (unique) users, 
#WAU — the number of weekly active users, 
#MAU — the number of monthly active users

#To calculate daily, weekly and monthly activity, we'll first create separate columns for year, month, and week values.
visits['date'] = pd.to_datetime(visits['Start Ts']).dt.date
visits['month'] = visits['Start Ts'].astype('datetime64[M]')
visits['week']  = visits['Start Ts'].astype('datetime64[W]')
orders['month'] = orders['buy_ts'].astype('datetime64[M]')
orders['week'] = orders['buy_ts'].astype('datetime64[W]')

visits.head()

In [ ]:
#Now let's calculate metrics. We'll group the data by session date/week/month and find the means:
dau=visits.groupby(visits['date'])['Uid'].nunique().reset_index()
#Calculating dau
dau_total = visits.groupby('date').agg({'Uid':'nunique'}).mean()

#Calculating wau
wau_total = visits.groupby('week').agg({'Uid':'nunique'}).mean()

#Calculating mau
mau_total = visits.groupby('month').agg({'Uid':'nunique'}).mean()

#Calculating sticky factors
sf_w = dau_total / wau_total
sf_m = dau_total / mau_total
#Printing the results
print('Average number of daily users:', int(dau_total))
print('Average number of weekly users:',int(wau_total), 'sticky factor:', round(float(sf_w),2))
print('Average number of monthly users:',int(mau_total), 'sticky factor:', round(float(sf_m),2))

We see that in average there are 907 users who use the product every day. Only 16% of the users who have used the product at least once a week, continue to do it every day, and only 4% of the users who have used the product at least once a month, continue to use it every day. 25% of users have used the service once a month us it weekly. Not so many users like to use the product regularly, may be we need to find the way how to involve more people to our product.
Let' look how dau, wau and mau change in time. We will plot line graph and mark the mean value to see the dynamic of daily, weekly and monthly users

In [ ]:


fig = px.line(dau,x="date", y="Uid",title='DAU')
#adding reference line with average DAU over time
fig.add_hline(y=dau['Uid'].mean(),line_dash="dash", line_color="purple", annotation_text="average DAU",
             annotation_position="top left")
fig.show()

DAU is stable. There is sudden peak in one day the dau is 3319, but it was only one day, after that dau returned to its original value.
In the same time we see that wau changes in time significantly. We have increas from 2000 users in August 2017 to 7800 users in October 2017. And we have this level stable till April 2018, after that date dau began to decrease. That means that our sticky factor is falling down.Less users used the product daily. For wau we have obviously the same peak of 10586 users. 

In [ ]:
wau=visits.groupby(visits['week'])['Uid'].nunique().reset_index()
wau.head()



In [ ]:

fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text="WAU")))
fig.add_trace(go.Scatter(x=wau['week'], 
                        y=wau['Uid'],
                    mode='lines+markers',
                    name='lines+markers',line = dict(color='purple', width=5)))
fig.add_hline(y=wau['Uid'].mean(),line_dash="dash", line_color="red", annotation_text="average WAU",
             annotation_position="top left")
fig.show()

In [ ]:
mau=visits.groupby(visits['month'])['Uid'].nunique().reset_index()
mau.head()


In [ ]:
print(mau['Uid'].mean())

In [ ]:

fig = go.Figure(layout=go.Layout(
        title=go.layout.Title(text="MAU")))
fig.add_trace(go.Scatter(x=mau['month'], 
                        y=mau['Uid'],
                    mode='lines+markers',
                    name='lines+markers',line = dict(color='purple', width=5)))
fig.add_hline(y=mau['Uid'].mean(),line_dash="dash", line_color="red", annotation_text="average MAU",
             annotation_position="top left")
fig.show()

Same dynamic and correlation between WAU and MAU with periods when MAU increases more than WAU (periods when users comes to the site with less regularly).

2.How many sessions are there per day? (One user might have more than one session.) Let's find number of sessions per each user. We can count the sessions and then we can divide number of sessions on users.



In [ ]:
#Creating the session table
sessions = visits.groupby('date').agg({'Uid':['count', 'nunique']})

#Renaiming the columns
sessions.columns = ['n_sess', 'n_user']
sessions['sess_per_user'] = sessions.n_sess / sessions.n_user

#Plotting line graph to see the dynamic of session per user
fig, ax = plt.subplots(figsize = (10,6))
ax.grid(axis ='y')
ax.plot(sessions.index, sessions['sess_per_user'], label = 'Sessions per user')

#Adding h-line with mean number of session per user
plt.axhline(y=sessions['sess_per_user'].mean(), linewidth=1, color = 'blue', alpha = 0.7)
mean_sess_per_user = sessions['sess_per_user'].mean()
ax.annotate(f"{mean_sess_per_user:.2f}", 
             xy=(sessions.index[0], mean_sess_per_user), xytext=(-30, 5), 
             textcoords="offset points", ha="right", va="top", color='blue', fontsize=10)


#Adding additional info
ax.legend()
ax.set_xlabel('Date')
ax.set_ylabel('Number of sessions')
ax.set_title('Number of sessions per user')
plt.show()



Users open the site just once a day. Number of session is stable throughout all the period. However there are one high peak 1.23 sessions per user and one low peak with 1.0 session.


In [ ]:
#3.Length of each session. Average session length, or ASL, is the amount of time users spend with a product in the average session.
visits['session_duration_sec'] = (visits['End Ts'] - visits['Start Ts']).dt.seconds
print(visits['session_duration_sec'].mean()) 

In [ ]:
#let's have a look at the distribution
visits['session_duration_sec'].hist(bins=50) 

In [ ]:
#When the distribution is normal or close, it's correct to use the mean or median. But in our case it's impossible, so we have to calculate the mode:
asl=visits['session_duration_sec'].mode()
#Printing the result
print('The average  daily session length per user (ASL) is:', int(asl), 'seconds')

# 4.User's retention rate. 

The retention rate tells how many users from a cohort have remained active compared to their initial number.

Basically, for retention we need to find the differnce between any session and first session.

In [ ]:
#Find first activity date for each user and join it into the visits dataset
first_activity_date=visits.groupby('Uid')['date'].min()
first_activity_date.name='first_activity_date'
visits=visits.join(first_activity_date, on ='Uid')
#Check
visits.head()

In [ ]:
#Sett the correct type of `first_activity_date` column

visits['first_activity_date']= pd.to_datetime(visits['first_activity_date'])


In [ ]:
#Add `first_activity_month` column

visits['first_activity_month']=(visits.first_activity_date - pd.to_timedelta(
    visits.first_activity_date.dt.day, unit='d')+ timedelta(days=1)).dt.date
#Check
visits.head()

In [ ]:
#Adding `activity_month` column

visits['date']=pd.to_datetime(visits['date'])
visits['activity_month']=(visits.date-pd.to_timedelta(
    visits.date.dt.day, unit='d')+timedelta(days=1)).dt.date
visits.head()

In [ ]:
#Calculate cohort lifetime
visits['cohort_lifetime']=((visits['activity_month']-visits['first_activity_month'])/np.timedelta64(1,'M')).round().astype(int)
visits.head()

We can group the visits table with first activity month and cohort lifetime and calculate the number of unique users in each lifetime week for each cohort.
After we divide the number of users in lifetime month to the number of users in first activity month and get the Retention rate.

In [ ]:
#Create cohort table
cohorts=visits.groupby(['first_activity_month', 'cohort_lifetime']).agg({'Uid':'nunique'}).reset_index()
#Initial number of users for the cohort
initial_user_count=cohorts[cohorts['cohort_lifetime']==0][['first_activity_month', 'Uid']]
#Rename the column
initial_user_count=initial_user_count.rename(columns={'Uid':'cohort_users'})
#Merge with cohort users
cohorts=cohorts.merge(initial_user_count, on='first_activity_month')

cohorts.head()


In [ ]:
#Add retention rate column
cohorts['retention']=cohorts['Uid']/cohorts['cohort_users']
#Retention pivot table
retention= cohorts.pivot_table(
    index='first_activity_month',
    columns='cohort_lifetime',
    values='retention',
    aggfunc='sum'
)
retention.head()


Plot the gradient heatmap of the retention rates 

In [ ]:
sns.set(style='dark')
plt.figure(figsize=(11, 7))# Set the figure size
plt.title('Retention rate')# Set the visualization title
sns.heatmap(
    retention,
    annot=True,
    fmt='.1%',
    linewidths=1,
    linecolor='red',
    vmax=0.1
) # Make a heatmap

The retention rate is small, less then 10% of users continue using the product after the first month.

The most successful cohort is the first one June 2017 Cohort. They have 7.9% of users who continue to use the site in the next month after the first usage and the have that rate for the next 6 months. Then after 6 months retention rate begin to drop to 4.5% in May 2018. 

The September 2017 Cohort gas the highest retention rate in the first month 8.5%, but it dropped rapidly in 3 months less than 4%. We see also that from December 2017 and in 2018 retention rate begins to drop for those cohorts below 6% in the first lifetime month.

We have 907 users, who use the product every day in average. 

Average number of weekly users - 5716

Average number of monthly users - 23228. 

Only 16% of the users who have used the product at least once a week, continue to do it every day, and only 4% who have used the product at least once a month, continue to use it every day. 

25% of users who have used the site once a month use it weekly. We see that WAU changes in time significantly. We have increase from 2000 users in August 2017 to 7800 users in October 2017. And we have this level stable till April 2018, after that DAU began to decrease.

That means that our sticky factor is falling down. In average it is 16%, but for the period form October 2017 till April 2018 it is just 12%.

Most often users spend just 1 minute using the product, but the half of the total number of users spend more than 6 minutes during the one session.

Less then 10% of users continue using the product after the first month when they began to use it. 

The September Cohort has the highest retention rate in the first month (8.5%) but it dropped at 3 months to 4%. The most successful cohort is the June 2017 Cohort. 

From December 2017 to 2018 retention rate begins to drop for those cohorts to 6% in the first lifetime month. My be there is a influence by the seasonal factor.

# 2. Sales


When do people start buying? Conversion depicts how long does it take from first visit to the purchase in days.


1.Let's groupby on orders table in order to find the first purchase for each user. First order df with users that've made purchases and the date of their first order.

2.Find the first visit for each user. 

3.Merge two dataframes together: user,first visit, first order.

4.Find the difference in days between the first order and first visit

In [ ]:
#Find first purchase date for each user
first_purchase_date = orders.groupby('Uid')['buy_ts'].min()
#Renaming the column
first_purchase_date.name = 'first_purchase_date'

#Add the `first_purchase` column to orders table
orders = orders.join(first_purchase_date, on = 'Uid')

#Check
orders.sample(5)

In [ ]:
#Adding the `first_activity_date` column to orders table
orders = orders.merge(first_activity_date, on = 'Uid')
#Adjusting format of `first_activity_date` column
orders['first_activity_date'] = pd.to_datetime(orders['first_activity_date'])

#Create `purchase_time` column
orders['purchase_time'] = (
    (orders.first_purchase_date - orders.first_activity_date)/np.timedelta64(1, 'D')).astype('int')

#Plot a histogram,
orders['purchase_time'].hist(bins= 30)
plt.title('Conversion')
plt.xlabel('Days')
plt.ylabel('Number of users')
plt.show()
print('Median conversion value is', orders['purchase_time'].median(), 'Days')

In [ ]:
orders['purchase_time'].describe()


We see that people usually order something in the same day. 75% of people make the first order within 4 days. Now let's find how many people convert at all. 

In [ ]:
#how many visits converted
print(' The overall conversion is {:.1%}'.format(orders['Uid'].nunique()/visits['Uid'].nunique()))

Any way, at least 16% of users made  one order or more.

2.How many orders do they make during a given period of time?


In [ ]:
#defining cohort month through first purchase
#Creating `first_buy_month`  and `buy_month` columns
orders['first_buy_month'] = (orders.first_purchase_date - pd.to_timedelta(
    orders.first_purchase_date.dt.day, unit='d') + timedelta(days=1)).dt.date
orders['buy_month'] = (orders.buy_ts - pd.to_timedelta(
    orders.buy_ts.dt.day, unit='d') + timedelta(days=1)).dt.date
#Creating `cohort_lifetime`column
orders['cohort_lifetime'] = ((
    orders['buy_month'] - orders['first_buy_month']) / np.timedelta64(1,'M')).round().astype('int')
   
#Check
orders.sample(5)


First_order_month now defines our cohort. Now let's aggregate the data : for each cohort we will find the month of purchase (then age from that), the number of purchases and unique customers. Calculate orders_per_buyer for each cohort and each month

In [ ]:
#let's define cohort size. this is one of the most important aggregation that we will use in cohort analysis
#using the month of first purchase we will define which cohort buyer belongs to
cohorts_buy = orders.groupby(['first_buy_month', 'cohort_lifetime']).agg({'Uid':['count','nunique']}).reset_index()
cohorts_buy.columns = ['first_buy_month', 'cohort_lifetime', 'n_orders', 'n_buyers']
#Calculating the initial number of users for each cohort
init_uid_buy = cohorts_buy[cohorts_buy['cohort_lifetime']==0][['first_buy_month', 'n_buyers']]
#Renaming the column for initial number of cohort buyers
init_uid_buy=init_uid_buy.rename(columns={'n_buyers':'cohort_count'})

#Adding the column `cohort_count` and calculating `orders_per_buer` in cohorts_buy table
cohorts_buy = cohorts_buy.merge(init_uid_buy, on='first_buy_month')
cohorts_buy['orders_per_buyer'] = cohorts_buy.n_orders / cohorts_buy.cohort_count

#Creating pivot
cohorts_buy_pivot = cohorts_buy.pivot_table(
    index='first_buy_month', columns='cohort_lifetime', values = 'orders_per_buyer', aggfunc='sum')

#Checking the result
cohorts_buy_pivot.round(2).fillna('-')

In [ ]:
#which cohort will be outrunning the others in term of cumulative purchases per user.
#pivot
cohorts_buy_pivot = cohorts_buy.pivot_table(
    index='first_buy_month', columns='cohort_lifetime', values = 'orders_per_buyer', aggfunc='sum').cumsum(axis=1)

#Check
cohorts_buy_pivot.round(2).fillna('-')

We see that our first cohort has the biggest number of orders per user per month. 2.19 orders per user in total.

3.What is the average purchase size?
We can make a pivot table to find this value.
We create a pivot and plot a graph for total mean purchase size


In [ ]:
#Calculating average order size
order_size = orders.pivot_table(
    index='first_buy_month', columns = 'cohort_lifetime', values = 'revenue', aggfunc='mean')

#Adding the totals
order_size.loc['Mean'] = order_size.mean(numeric_only=True, skipna=True, axis=0)
order_size = pd.concat([order_size, order_size.mean(numeric_only=True, skipna=True, axis=1).rename('Mean')], axis=1)

#Printing the result
order_size.round(2).fillna('-')

In [ ]:
#Revenue distribution by month:
revenue_dist = orders.groupby('buy_month')['revenue'].mean().reset_index()

#Line graph 
fig, ax = plt.subplots(figsize=(14,10))
ax.plot(revenue_dist['revenue'])
ax.grid(axis='y')

#h-line
revenue_mean = orders.revenue.mean()
plt.axhline(y=revenue_mean, linewidth=3, color = 'red', alpha = 0.3)
ax.text(revenue_dist['revenue'].iloc[0], revenue_mean-0.25, f"Mean: {revenue_mean:.0f}", color='red')

#graph labels
plt.xlabel('Month')
plt.ylabel('Dollar')
plt.suptitle('Mean revenue')
plt.show()

<div class="alert alert-block alert-info">
<b> On historical data, we see the following picture of the change in average revenue on a monthly basis:

Firstly the average revenue increases and varies in a small range around the mean( about 5 USD), but tends to decrease towards the end of the study period.</b> <a class="tocSkip"></a>
</div> 

# 4.How much money do they bring? (LTV)

When a customer buys for the first time, we don't know if he will repeat the purchase and become a regular, or immediately leave. 
If you know the entire customer journey, then you can rationally distribute the budget: do not spend it on disloyal customers and invest more in those who stay for a long time. Lifetime Value (LTV) helps to predict this.

In [ ]:
#get the revenue per cohort in each month
ltv_cohort=orders.groupby(['first_buy_month','month'])['revenue'].sum().reset_index()
ltv_cohort=ltv_cohort.merge(init_uid_buy, on='first_buy_month')
ltv_cohort['first_buy_month']=ltv_cohort['first_buy_month'].astype('datetime64[M]')
ltv_cohort['age']=((ltv_cohort['month'] - ltv_cohort['first_buy_month']) / np.timedelta64(1,'M')).round()
#Calculating the evarage revenue per user
ltv_cohort['ltv'] = ltv_cohort.revenue / ltv_cohort.cohort_count
ltv_cohort

In [ ]:
ltv_cohort_piv=ltv_cohort.pivot_table(
    index='first_buy_month', 
    columns='age', 
    values='ltv', 
    aggfunc='sum'
).cumsum(axis=1)

In [ ]:
# Creating LTV pivot table
ltv_cohort_piv.index=ltv_cohort_piv.index.astype(str)


#Plotting heat map
plt.figure(figsize=(12,8))

sns.heatmap(ltv_cohort_piv, annot=True, fmt='.2f', linewidths=1, linecolor='grey', cbar_kws= {'orientation': 'vertical'} 
            ).set(title ='LTV')

plt.show()

As we see, the best is September 2017 cohort. Each user from this cohort gives us 13.44 usd. 
June 2017 Cohort has also good with 11.88 usd. Users from other cohorts brings less than 9 usd. The worst result has the new fresh cohort of June 2018.


# 3.Marketing. Reports and metrics




# How much money was spent? Overall, per source and over time.


In [ ]:
#Let's find total costs.
print('Total marketing cost is {} usd'.format(costs['costs'].sum()))

In [ ]:
#Build a graph showing marketing costs of different ad sources
#Group costs
costs_d = costs.groupby('source_id')['costs'].sum().reset_index()

#Plot a bar
costs_d.plot(kind = 'bar', x = 'source_id', y = 'costs', figsize = (9,5)).grid(axis = 'y')
plt.ylabel('USD')
plt.title('Costs per different ad sources, USD')
plt.show()

#Print sorted costs
costs_d.sort_values('costs', ascending=False)


We spend 141 321 usd for acquisition through the source 3. Now let's build a graph showing costs of different sources spent through time.

In [ ]:
#Groupcosts with `source` column
costs_source = costs.groupby(['source_id', 'dt'])['costs'].sum().reset_index()

#Plot linegraph for the most expansive source
plt.figure(figsize = (15,10))
ax = sns.lineplot(
        data=costs_source.query('source_id==3'), 
        x='dt', y='costs', hue='source_id', linewidth=1,
        palette=sns.color_palette("bright", 1)
     )
ax.grid(axis='y')
ax.set_title('Costs Sorce N3')
ax.set_xlabel('Months')
ax.set_ylabel('Dollars')
plt.show()

In [ ]:
#Plot other sources
plt.figure(figsize = (15,10))
ax = sns.lineplot(
        data=costs_source.query('source_id!=3'), 
        x='dt', y='costs', hue='source_id', linewidth=1,
        palette=sns.color_palette("bright", 6)
     )
ax.grid(axis='y')
ax.set_title('Other sources')
ax.set_xlabel('Months')
ax.set_ylabel('Dollars')
plt.show()

In [ ]:
#costs over time
costs.set_index('dt', inplace=True)
costs_month = costs['costs'].resample('M').sum()

In [ ]:

#Plot the bar graph
costs_month.index = costs_month.index.strftime('%y-%m')
costs_month.plot(kind = 'bar', x = 'dt', y = 'costs', figsize = (10,6)).grid(axis = 'y')
plt.ylabel('USD')
plt.title('Costs over time, USD')
plt.xlabel('Time')
plt.show()

As it seen on the graph the marketing cost grow up from August 2017 untill the December of 2017. In January 2018 them began to decrease. 
The retention rate began to increase in that period of time synchronously, obviously as a result of spending more money on user's acquisition. After the decreasing of costs in January 2018 the retention rate also decrease.

# How much did customer acquisition from each of the sources cost?


In [ ]:
costs = costs.reset_index()
costs['dt_month'] = (costs['dt'] - pd.to_timedelta(costs['dt'].dt.day, unit='d') + timedelta(days=1)).dt.date

#Group costs
costs_month = costs.groupby('dt_month')['costs'].sum().reset_index()

#Check
costs_month

In [ ]:
#buyers_month table
buyers_month = orders.groupby('first_buy_month')['Uid'].nunique().reset_index()
#Rename the columns
buyers_month.columns = ['dt_month', 'n_buyers']
#Check
buyers_month

In [ ]:
# Merge the both tables
costs_month = costs_month.merge(buyers_month, on='dt_month')
#Calculating CAC
costs_month['cac'] = costs_month.costs / costs_month.n_buyers
costs_month.head()

In [ ]:
#Plot the graph
plt.figure(figsize = (10,6))
ax = sns.lineplot(data=costs_month, x='dt_month', y='cac', linewidth=1, palette=sns.color_palette("bright", 1))
ax.grid(axis='y')

#mean CAC
cac_mean = float(costs_month['cac'].mean())
#Plot line with the mean CAC
plt.axhline(y=cac_mean, linewidth=1, color = 'blue', alpha = 0.7)
ax.text(costs_month['dt_month'].iloc[0], cac_mean +0.05, f"Mean CAC: {cac_mean:.0f}", color='blue', va='center')

#Plot info
ax.set_title('Costs of user acquisition in time')
ax.set_xlabel('Months')
ax.set_ylabel('USD')
plt.show()

<div class="alert alert-block alert-info">
<b>
The Mean CAC per user is about 9 USD. This value changes significantly over time. For example, it reaches a maximum value in August 2017 of more than 10 and a minimum in May 2018 of more than 7. In general, acquisition seems prohibitively expensive, given that Yandex itself is engaged in contextual advertising. We need to think about properly built SEO optimization of this site, although this is beyond the scope of our study, since the sources of attraction for us are marked only with numbers.
</b> <a class="tocSkip"></a>
</div> 


# How much did customer acquisition from each of the sources cost?

CAC is the cost of customer acquisition. To calculate CAC, we need to divide marketing costs by the number of customers attracted. Let's calculate CAC for different channels of ad.

In [ ]:
#table with the first source for each user
user_source  = visits.sort_values(['Start Ts']).groupby('Uid')['source_id'].first().reset_index()

#Merge user_sorce to orders table
orders = orders.merge(user_source, on = 'Uid')

#Group orders table with buy_date and source_id to find number of users per each source
buyers_month_source = orders.groupby(['first_buy_month', 'source_id'])['Uid'].nunique().reset_index()



In [ ]:
#cost_month_source table
cost_month_source = costs.groupby(['dt_month', 'source_id'])['costs'].sum().reset_index()

#Rename columns to merge
buyers_month_source.rename(columns={'first_buy_month':'dt_month', 'Uid': 'n_buyers'}, inplace=True)
#Merge
cost_month_source = cost_month_source.merge(
    buyers_month_source, how='left', left_on = ['dt_month', 'source_id'], right_on = ['dt_month', 'source_id'])
#CAC per source
cost_month_source['cac_source'] = cost_month_source.costs / cost_month_source.n_buyers

#Check
cost_month_source.head()

In [ ]:
#Plot line graph
plt.figure(figsize = (10,6))
ax = sns.lineplot(data=cost_month_source, x='dt_month', y='cac_source',
                  hue = 'source_id', linewidth=1, palette=sns.color_palette("bright", 7))
ax.grid(axis='y')

#Plot info
ax.set_title('CAC of different sources in time')
ax.set_xlabel('Months')
ax.set_ylabel('USD')
plt.show()


In [ ]:
#Let's also calculate overall average CAC per source

cac_source_mean = (cost_month_source.groupby('source_id')['cac_source'].mean()
                   .sort_values(ascending = False).reset_index()
                  )
cac_source_mean

Average CAC is the most expensive for source 3, it has high peak in August, and dropped greatly in May. 

The same dynamic have sources 2,1,5,4 and they are more stable in this period of time. 

Source 9 have the other dynamics, it increase in May to 8.5 usd. 

Very unstable CAC for source 10. Average CAC is 4.9 usd and it changes in range from 3 to 8 usd.

# How worthwhile where the investments? (ROI)
ROI is the percentage ratio between income and investment in a business. The higher is the percentage, the more effective is the investment.

In [ ]:
#let's start with ROI per cohort.
costs_month_roi = costs_month[['dt_month', 'cac']]
#Renaming column `dt_month` in order to merge
costs_month_roi.columns=['first_buy_month', 'cac']

costs_month_roi['first_buy_month']=costs_month_roi['first_buy_month'].astype('datetime64[M]')

#merging the tables costs_month_roi and ltv_cohort
roi = ltv_cohort.merge(costs_month_roi, on = 'first_buy_month')

#Check
roi.head()

In [ ]:
#Calculate ROI
roi['roi'] = roi.ltv / roi.cac


In [ ]:
#pivot of roi
roi_pivot = roi.pivot_table(index='first_buy_month', columns = 'age', values = 'roi', aggfunc = 'mean'
                           ).cumsum(axis = 1)


In [ ]:
# the heatmap of ROI
sns.set(style='dark')
plt.figure(figsize=(11,6))
sns.heatmap(roi_pivot, annot=True, fmt='.2f', linewidths=1, vmax=1.4, linecolor='gray')
plt.title('Month Cohorts: ROI')
plt.show()

We see that only 2 cohorts are partly profitable. First June 2017 cohort is payed off from the 7th month so in December 2017 and finally it has 33% of gross profit for a year.

The most profitable is September 2017 cohort with the biggest check. It is profitable from the 4th month and it has gross profit of 42%. Costs for all other cohorts are bigger than revenue. So we would recommend to increase marketing costs as there is a correlation between those costs and retention rate that we have determined.

In [ ]:
#Let's find ROI per source

#Group orders to find revenue per source
ltv_source = orders.groupby(['source_id'])['Uid', 'revenue'].agg({'Uid': 'nunique', 'revenue' :'sum'}).reset_index()
#Renaming the columns
ltv_source.rename(columns = {'Uid':'n_buyers'}, inplace = True)

#ltv
ltv_source['ltv_source'] = ltv_source.revenue / ltv_source.n_buyers

#Check
ltv_source


In [ ]:
#Merging ltv_source and costs_d tables with `source_id` column
roi_source = ltv_source.merge(costs_d, on = 'source_id')

#Calculating CAC per source 
roi_source['cac_source'] = roi_source.costs / roi_source.n_buyers

#Calculating ROI
roi_source['roi'] = roi_source.ltv_source / roi_source.cac_source
 
#Checking the result
roi_source

In [ ]:
#Plotting bar graph
plt.figure(figsize = (10,6))
ax = sns.barplot(data=roi_source, x='source_id', y='roi',linewidth=1, palette=sns.color_palette("bright", 1))
ax.grid(axis='y')
plt.axhline(y=1, linewidth=3, color = 'black', alpha = 0.7, linestyle = '--')

#Plotting additional info
ax.set_title('ROI of different sources')
ax.set_xlabel('Source')
ax.set_ylabel('roi coefficient')
plt.show()

ROI per source. 

The profitable sources are only 1,2 5,9. The biggest most expensive source number 3 is ruinously unprofitable, only about 40% of marketing costs are payed off. So should advise to redistribute marketing budjet in profitable sources in order to get more profitable users and stop burning money in source 3.

# Conclusion 



Yandex.Afisha task was to help optimize marketing expenses. 

We had server logs with data on Yandex.Afisha visits from June 2017 through May 2018, dump file with all orders for the period and marketing expenses statistics. 

Purpose of the analysis:

find out how people use the product, when they start to buy, how much money each customer brings to the company, when they pay off.

We got data from 3 tables: visits, orders and costs. There were no missing values we just changed some column names to lowercase with the use of underscore between the words, we changed type of time stamp columns in order to work with the dates properly. We optimize type of device to category type.

Yandex.Afisha has 907 users, who use the product every day in average. 
Average number of weekly users - 5716
Average number of monthly users - 23228. 
16% of the users who have used the product at least once a week, continue to do it every day, and only 4% of the users who have used the product at least once a month, continue to use it every day. 

25% of users who have used the service once a month us it weekly. We saw that users open the product just once a day. Most often users spend just 1 minute using the product, but the half of the total number of users spend more than 6 minutes during the one session.

10% of users continue using the product after the first week when they began to use it. The September 2017 Cohort has the highest Retention rate in the first month, but it dropped. In 3 months it drops to 4%. The most successful cohort is the first one June 2017 Cohort. They have 7.9% of users who continue to use the site in the next month after the first usage and the have this retention rate for the next 6 months. Then after 6 months retention rate begin to drop to 4.5% in May 2018. We see also that from December 2017 and in 2018 retention rate begins to drop for those cohorts below 6% in the first lifetime month.

Clients usually order something in the same day. 75% of people make the first order within 4 days. But only 16% of users made at least one order.

We discovered that our first cohort has the biggest number of orders per user per month. It has 2.19 orders per user in total. 
Average order size is 7.73 usd, Yandex.Afisha has the same average check in first cohort (6.85)
The September Cohort has the maximum average check of 17.29 usd. The December2017 cohort has the biggest check of 14 usd. 
Average check is growing for the first June cohort, But in last month it dropped to average value.

The September and December cohorts have the biggest average check in March (62.57 and 26.08 usd accordingly), then it decreases.

The best revenue gives September Cohort. Each user from this cohort gives Yandex.Afisha 13.44 usd. 

Marketing
The most expensive is the source number 3. Yandex.Afisha spent more than 140 000 usd for acquisition through that source. 
Marketing costs grew up till the end of 2017 year, after that the began to decrease. We have already notices that retention rate began to increase in that period of time. So this could be a result of spending more money on users acquisition. So there is a correlation between decreasing marketing costs and  decreasing retention rate.

The Mean CAC per user is about 9 USD. This value changes significantly over time. For example, it reaches a maximum value in August 2017 of more than 10 and a minimum in May 2018 of more than 7. In general, acquisition seems prohibitively expensive, given that Yandex itself is engaged in contextual advertising. We need to think about properly built SEO optimization of this site, although this is beyond the scope of our study, since the sources of attraction for us are marked only with numbers.The most expensive is the source number 3. Average CAC is 13.8 usd for it and this level of costs is mostly constant. We have only high peak in August, but then CAC dropped to the mean level and we have great decrease in May 2018 where CAC dropped to 11 usd per source. We have the same dynamic with source 2 which have an average CAC of 12.5 USD. The same dynamic have sources 1,5,4 and thy are more stable in time, but for Source 9 we have the other dynamics: stable level about 5.4 usd and sudden increase in May to 8.5 usd. We have very unstable CAC for source 10. Average CAC of that source is 4.9 usd but it changes in range from 3 to 8 usd.

Durin ROI analysis we saw that only 2 cohorts are payed off. Our first June cohort is payed off from the 7th month so in December 2017 and finally it has 33% of gross profit for a year. The most profitable is September cohort where was the biggest average check. It is profitable from the 4th month and it has gross profit of 42% for the whole analysis period. Costs for all other cohorts are bigger than profit. So we would recommend to increase marketing costs as there is a correlation between those costs and retention rate that we have determined during the analysis.

ROI per source. The profitable sources are only 1,2 5,9. The biggest most expensive source number 3 is ruinously unprofitable, only about 40% of marketing costs are payed off. So should advise to redistribute marketing budjet in profitable sources in order to get more profitable users and stop burning money in source 3. 

Unfortunately,  marketing is not ok, because only 3 ad sources and 2 cohorts are payed off and profitable. There is such correlation that the more money we spend to users acquisition the more users we get. We found out that marketing expenses began to decrease in 2018, and this result in decrease in number of users. 
We recommend to increase investments into marketing, but change the ad source №3 to the profitable sources 1, 2 and 9.